# Example of import_raster cmd using API of geonode_conabio and [kale](https://github.com/kubeflow-kale/kale) functionality

References:

https://github.com/CONABIO/geonode/blob/master/notebooks/towards_module_python_to_ingest_programmatically_into_geonode/1_normalizing_and_reprojecting.ipynb

https://github.com/CONABIO/kube_sipecam_playground/blob/master/MAD_Mex/notebooks/1_issues_and_nbooks/1_issue_8_pipeline_upload_lc_result_to_geonode_using_kale.ipynb

https://github.com/CONABIO/antares3/blob/develop/madmex/management/commands/db_to_raster.py



**Raster has small size**

In [1]:
import os
import glob
import json

import psycopg2
from dotenv.main import load_dotenv
from pyproj import Proj
from rasterio.merge import merge
import rasterio
from rasterio.warp import transform_geom
from rasterio import features
from docker import APIClient

from geonode_conabio.wrappers import reproj_and_write_one_band_raster
from geonode_conabio.utils_docker import import_layers_via_docker



# Reproject and write to FS

In [16]:
path = "/shared_volume/land_cover_results_parallel"

filenames = [f for f in glob.glob(os.path.join(path, "*.tif")) if not "geonode" in f and not "merge" in f]
output_filenames = [f.replace(".tif", "_processed_for_geonode.tif") for f in filenames]

for idx, filename in enumerate(filenames):
    with rasterio.open(filename) as src:
        src_crs = src.crs.to_string()
        proj_crs = Proj(src_crs)
        if not proj_crs.crs.is_geographic:
            reproj_and_write_one_band_raster(src, output_filenames[idx],
                                             is_geographic=False)
        else:
            reproj_and_write_one_band_raster(src, output_filenames[idx])

# Merge

In [17]:
src_files_to_mosaic = [rasterio.open(f) for f in output_filenames]
# Retrieve metadata of one file for later use
meta = src_files_to_mosaic[0].meta.copy()
mosaic, out_trans = merge(src_files_to_mosaic)
meta.update(width=mosaic.shape[2],
            height=mosaic.shape[1],
            transform=out_trans,
            compress="lzw")

# Mask

In [8]:
dotenv_path = load_dotenv("/shared_volume/.antares")

DB_DATABASE = os.getenv("DATABASE_NAME", "")
DB_USER = os.getenv('DATABASE_USER', "")
DB_PASSWORD = os.getenv('DATABASE_PASSWORD', "")
DB_HOST = os.getenv('DATABASE_HOST', "")
DB_PORT = os.getenv('DATABASE_PORT', "")

db_params = {'drivername': 'postgresql',
             'database': DB_DATABASE,
             'username': DB_USER,
             'host': DB_HOST,
             'port': DB_PORT,
             'password': DB_PASSWORD}

conn = psycopg2.connect(database=DB_DATABASE,
                        user=DB_USER,
                        password=DB_PASSWORD,
                        host=DB_HOST,
                        port=DB_PORT)

string1 = "SELECT public.madmex_region.name AS name, ST_AsGeoJSON(public.madmex_region.the_geom) FROM public.madmex_region WHERE name="

name = "'Chiapas';"

query = "".join([string1,name])

curs = conn.cursor()


curs.execute(query)


region_db = curs.fetchall()[0][1]
curs.close()
geometry_region = json.loads(region_db)


# rasterize region using mosaic as template
mask_array = features.rasterize(shapes=[(geometry_region, 1)],
                                out_shape=(mosaic.shape[1],mosaic.shape[2]),
                                fill=0,
                                transform=meta['transform'],
                                dtype=rasterio.uint8)

# Apply mask to mosaic
mosaic[:,mask_array==0] = 0

# Write merge to FS

In [22]:
# Write results to file
filename = "raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_merge_2.tif"
filename_mosaic = os.path.join(path, filename)

with rasterio.open(filename_mosaic, "w", **meta) as dst:
    dst.write(mosaic)
    
    

In [23]:
filename_mosaic

'/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_merge_2.tif'

# Upload to geonode

In [ ]:
region = "".join(["\"Chiapas, Mexico, North America, Latin America\""])
name = "\"Chiapas_lc_2017_landsat8_test merge3\""
title = "\"Land cover Chiapas landsat8 2017 merge3 test using kale\""
abstract = "\"Test merge3 kale\""
kw = "\"Chiapas\""


c = APIClient(base_url='tcp://172.17.0.1:1111')
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               filename_mosaic, " ",
               "--settings=geonode.local_settings"])
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)
ex_start = c.exec_start(exec_id=ex)

print(ex_start)
c.close()
#close & clean:
#os.remove(filename_mosaic)
#for idx, of in output_filenames:
#    src_files_to_mosaic[idx].close()
#    os.remove(of)

In [19]:

#for row in curs.fetchall():
#    #print(row)
#    region = row[1]

#curs.execute("""SELECT public.madmex_region.name AS name,
#ST_AsGeoJSON(public.madmex_region.the_geom) FROM public.madmex_region WHERE name='Chiapas';""")
